In [1]:
from env_vars import *
from flask import jsonify, request
from joblib import load, dump
from more_itertools import unique_everseen
import numpy as np
import pandas as pd
from pandas import json_normalize
import pickle
import psycopg2 as ps
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, Normalizer
import spotipy
import spotipy.util as util
import sys
import time

#### Token generation

In [2]:
token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)

### Application code from elastic beanstalk environment:
##### >Version label: app-200609_131842-8
##### >Source:20201624wO-sd_061020_v2.zip

#### Added time statements to Sound_drip methods for benchmark(s)

In [7]:
time_table = {'get_user_ids': [],
              'get_user_song_id_source_genre': [],
              'get_acoustical_features': [],
              'get_acoustical_features': [],
              'get_popularity': [],
              'get_artist_id': [],
              'get_genres': [],
              'create_feature_object': [],
              'get_results': [],
              'filter_model': [], 
              'song_id_prediction_output': [],
              'db_connect': [],
              'insert_user_predictions': [],
              'get_stale_results': [],
              'get_stale_seed': []}

if FLASK_ENV == 'production':
    db_table = 'recommendations'
elif FLASK_ENV == 'development':
    db_table = 'recommendations_dev'

class Sound_Drip:
    

    def __init__(self, token):
        self.token = token
        self.sp = spotipy.Spotify(auth=self.token)
        self.user_id, self.display_name = self.get_user_ids()
        self.stale_seed_list = self.get_stale_seed()
        self.stale_results_list = self.get_stale_results()
        self.song_id, self.source_genre = self.get_user_song_id_source_genre()
        self.acoustical_features = self.get_acoustical_features(self.song_id)
        self.popularity = self.get_popularity(self.song_id)
        self.song_features_df = self.create_feature_object(
            self.popularity, self.acoustical_features)
        self.results = self.get_results(self.song_features_df)
        self.filtered_list = self.filter_model(self.results, self.source_genre)
        self.song_id_predictions = self.song_id_prediction_output(
            self.filtered_list)
        self.insert_user_predictions()#, print("predicts inserted into db")

    def get_user_ids(self):
        
        print("START TIME FOR get_user_ids")
        start_time = time.time()
        current_user_dict = self.sp.current_user()
        display_name = current_user_dict['display_name']
        user_id = current_user_dict['id']
        #print("retrieving user id and display name for current token")
        print(f"TOTAL get_user_ids TIME: {time.time() - start_time} seconds")
        time_table['get_user_ids'].append(time.time() - start_time)
        return user_id, display_name

    def get_user_song_id_source_genre(self):
        
        print("START TIME FOR get_user_song_id_source_genre")
        start_time = time.time()
        stale_songs = self.stale_seed_list
        results = self.sp.current_user_saved_tracks(limit=50)
        for song_number in range(0, len(results['items'])):
            #print(song_number)
            song_id = results['items'][song_number]['track']['id']
            #print(song_id)
            if song_id not in stale_songs:
                artist_id = self.get_artist_id(song_id)
                genre = self.get_genres(artist_id)
                #print(genre)
                if genre != []:
                    break
                else:
                    continue
            else:
                if song_number == len(results['items']) - 1:
                    #print("application out of fresh seeds")
                    for song_id in stale_songs:
                        artist_id = self.get_artist_id(song_id)
                        genre = self.get_genres(artist_id)
                        if genre != []:
                            break
                        else:
                            continue
        print(f" TOTAL get_user_song_id_source_genre TIME: {time.time() - start_time} seconds")
        time_table['get_user_song_id_source_genre'].append(time.time() - start_time)
        return song_id, genre

    def get_acoustical_features(self, song_id):
        
        print("START TIME FOR get_acoustical_features")
        start_time = time.time()
        acoustical_features = self.sp.audio_features(song_id)[0]
        print(f" TOTAL get_acoustical_features TIME: {time.time() - start_time} seconds")
        time_table['get_acoustical_features'].append(time.time() - start_time)
        return acoustical_features

    def get_popularity(self, song_id):
        
        print("START TIME FOR get_popularity")
        start_time = time.time()
        popularity = self.sp.track(song_id)['popularity']
        print(f" TOTAL get_popularity TIME: {time.time() - start_time} seconds")
        time_table['get_popularity'].append(time.time() - start_time)
        return popularity

    def get_artist_id(self, song_id):
        
        print("START TIME FOR get_artist_id")
        start_time = time.time()
        artist = self.sp.track(song_id)['artists'][0]['id']
        print(f" TOTAL get_artist_id TIME: {time.time() - start_time} seconds")
        time_table['get_artist_id'].append(time.time() - start_time)
        return artist

    def get_genres(self, artist):
        
        print("START TIME FOR get_genres")
        start_time = time.time()
        genre = self.sp.artist(artist)['genres']
        print(f" TOTAL get_genres TIME: {time.time() - start_time} seconds")
        time_table['get_genres'].append(time.time() - start_time)
        return genre

    def create_feature_object(self, popularity, acoustical_features):
        
        print("START TIME FOR create_feature_object")
        start_time = time.time()
        popularity_dict = {'popularity': popularity}
        song_features = acoustical_features
        song_features.update(popularity_dict)
        song_features = {
            "audio_features": {
                key: song_features[key] for key in song_features.keys() & {
                    'popularity',
                    'acousticness',
                    'danceability',
                    'energy',
                    'instrumentalness',
                    'key',
                    'liveness',
                    'loudness',
                    'mode',
                    'speechiness',
                    'tempo',
                    'time_signature',
                    'valence'}}}

        df = pd.DataFrame.from_dict(
            json_normalize(
                song_features["audio_features"]),
            orient='columns')
        df = df.reindex(sorted(df.columns), axis=1)
        print(f" TOTAL create_feature_object TIME: {time.time() - start_time} seconds")
        time_table['create_feature_object'].append(time.time() - start_time)
        return df

    def get_results(self, song_features_df):
        
        print("START TIME FOR get_results")
        start_time = time.time()
        scaler = load("./models/scalar3.joblib")
        #print('Scaling data...')
        data_scaled = scaler.transform(song_features_df)
        normalizer = Normalizer()
        data_normalized = normalizer.fit_transform(data_scaled)
        #print('Loading pickled model...')
        model = load('./models/model5.joblib')
        results = model.kneighbors([data_normalized][0])[1:]
        #print('results returned')
        print(f" TOTAL get_results TIME: {time.time() - start_time} seconds")
        time_table['get_results'].append(time.time() - start_time)
        return results[0]

    def filter_model(self, model_results, source_genre_list):
        
        # loop takes KNN results and filters by source track genres
        print("START TIME FOR filter_model")
        start_time = time.time()
        #print(source_genre_list)
        #print("filter for genres initiated")
        genre_array = pickle.load(open("./data/genres_array_2.pkl", "rb"))
        filtered_list = []
        song_list_length = 20
        stale_results = self.stale_results_list
        model_results_before = len(model_results[0][1:])
        model_results = [index for index in model_results[0]
                         [1:] if index not in stale_results]
        model_results_final = model_results_before - len(model_results)
        #print(f'{model_results_final} stale tracks were removed for the user')
        for output_song_index in model_results:
            output_genre_list = genre_array[output_song_index]
            for output_genre in output_genre_list:
                output_genre = output_genre.strip(" ")
                for source_genre in source_genre_list:
                    source_genre = "'" + source_genre + "'"
                    if source_genre == output_genre:
                        filtered_list.append(output_song_index)
                    else:
                        continue
        filtered_list = list(unique_everseen(filtered_list))
        if len(filtered_list) >= song_list_length:
            #print("filter found at least 20 genre matches")
            filtered_list = filtered_list[0:20]
        else:
            counter = song_list_length - len(filtered_list)
            #print("length of filtered list:", len(filtered_list))
            #print(f'need to add {counter} items to final song output')
            for output_song_index in model_results:
                if output_song_index not in filtered_list:
                    if counter > 0:
                        filtered_list.append(output_song_index)
                        counter -= 1
                    else:
                        break
        #print(
        #    f"filtered list with {len(filtered_list)} unique song indices returned")
        print(f" TOTAL filter_model TIME: {time.time() - start_time} seconds")
        time_table['filter_model'].append(time.time() - start_time)
        return filtered_list

    def song_id_prediction_output(self, filtered_list):
        
        print("START TIME FOR song_id_prediction_output")
        start_time = time.time()
        similar_songs = []
        song_id_list = []
        #print('song_id_list loading...')
        song_id_array = pickle.load(open('./data/song_id_array3.pkl', 'rb'))
        #print('song_id_list loaded')
        for song_row in filtered_list:
            song_id = song_id_array[song_row]
            similar_songs.append({'similarity': [.99], 'values': song_id})
            song_id_list.append(song_id)
        song_result_output_dict = {"songs": similar_songs}
        song_id_and_index_dict = {
            song_id: song_index for song_id,
            song_index in zip(
                song_id_list,
                filtered_list)}
        #print("Results returned")
        print(f" TOTAL song_id_prediction_output TIME: {time.time() - start_time} seconds")
        time_table['song_id_prediction_output'].append(time.time() - start_time)
        return song_result_output_dict, song_id_and_index_dict

    def db_connect(self):
        
        print("START TIME FOR db_connect")
        start_time = time.time()
        conn = ps.connect(host=POSTGRES_ADDRESS,
                          database=POSTGRES_DBNAME,
                          user=POSTGRES_USERNAME,
                          password=POSTGRES_PASSWORD,
                          port=POSTGRES_PORT)
        cur = conn.cursor()
        print(f" TOTAL db_connect TIME: {time.time() - start_time} seconds")
        time_table['db_connect'].append(time.time() - start_time)
        return conn, cur

    # def get_user_ids(self):
    #     '''
    #     Retrieves user id from Spotfiy API
    #     Returns user_id, and display_name (display_name is for the database)
    #     '''
    #     current_user_dict = self.sp.current_user()
    #     display_name = current_user_dict['display_name']
    #     user_id = current_user_dict['id']
    #     print("retrieving user id and display name for current token")
    #     return user_id, display_name

    def insert_user_predictions(self):
        
        print("START TIME FOR insert_user_predictions")
        start_time = time.time()
        try:
            conn, cur = self.db_connect()
            for song_id, song_index in self.song_id_predictions[1].items():
                cur.execute(
                    f'INSERT INTO {db_table}'
                    '(userid,songid,songlistindex,seedsongid,recdate)'
                    f' VALUES (\'{self.user_id}\',\'{song_id}\',\'{song_index}\',\'{self.song_id}\',current_timestamp);')
            conn.commit()
            conn.close()
        except ps.DatabaseError as e:
            print(f'Error {e}')
            sys.exit(1)
        finally:
            if conn:
                conn.close()
        print(f" TOTAL insert_user_predictions TIME: {time.time() - start_time} seconds")
        time_table['insert_user_predictions'].append(time.time() - start_time)

    def get_stale_results(self):
        
        print("START TIME FOR get_stale_results")
        start_time = time.time()
        try:
            conn, cur = self.db_connect()
            query = f'SELECT DISTINCT (songlistindex) FROM {db_table} WHERE userid = \'{self.user_id}\';'
            cur.execute(query)
            query_results = cur.fetchall()
            stale_results_list = [index[0] for index in query_results]
        except ps.DatabaseError as e:
            print(f'Error {e}')
            sys.exit(1)
        finally:
            if conn:
                conn.close()
        print(f" TOTAL get_stale_results TIME: {time.time() - start_time} seconds")
        time_table['get_stale_results'].append(time.time() - start_time)
        return stale_results_list

    def get_stale_seed(self):
        print("START TIME FOR get_stale_seed")
        start_time = time.time()
        try:
            conn,cur = self.db_connect()
            query = f'SELECT DISTINCT (seedsongid) FROM {db_table} WHERE userid = \'{self.user_id}\' AND seedsongid is not null;'
            cur.execute(query)
            query_results = cur.fetchall()
            stale_results_list = [index[0] for index in query_results]
        except ps.DatabaseError as e:
            print(f'Error {e}')
            sys.exit(1)
        finally:
            if conn:
                conn.close()
        print(f" TOTAL get_stale_seed TIME: {time.time() - start_time} seconds")
        time_table['get_stale_seed'].append(time.time() - start_time)
        return stale_results_list


In [8]:
sd = Sound_Drip(token)

START TIME FOR get_user_ids
TOTAL get_user_ids TIME: 0.16990399360656738 seconds
START TIME FOR get_stale_seed
START TIME FOR db_connect
 TOTAL db_connect TIME: 0.8229291439056396 seconds
 TOTAL get_stale_seed TIME: 1.0671241283416748 seconds
START TIME FOR get_stale_results
START TIME FOR db_connect
 TOTAL db_connect TIME: 0.7213048934936523 seconds
 TOTAL get_stale_results TIME: 0.9598472118377686 seconds
START TIME FOR get_user_song_id_source_genre
START TIME FOR get_artist_id
 TOTAL get_artist_id TIME: 0.07767033576965332 seconds
START TIME FOR get_genres
 TOTAL get_genres TIME: 0.08097171783447266 seconds
 TOTAL get_user_song_id_source_genre TIME: 0.301846981048584 seconds
START TIME FOR get_acoustical_features
 TOTAL get_acoustical_features TIME: 0.08449816703796387 seconds
START TIME FOR get_popularity
 TOTAL get_popularity TIME: 0.08284544944763184 seconds
START TIME FOR create_feature_object
 TOTAL create_feature_object TIME: 0.000997304916381836 seconds
START TIME FOR get_res

#### INVESTIGATE this error message in cli
"""FutureWarning: Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False). warnings.warn("Method cleanup(connection_file=True) is deprecated, use cleanup_resources(restart=False).""""

In [ ]:
time_table 

In [ ]:
time_table.items()

In [ ]:
my_dict = {'foo':'bar'}
my_dict['foo']

In [ ]:
a = list(sd.song_id_predictions[1].items())

In [ ]:
'INSERT INTO {db_table} (userid,songid,songlistindex,seedsongid,recdate)'
' VALUES (\'{self.user_id}\',\'{song_id}\',\'{song_index}\',\'{self.song_id}\',current_timestamp);'

In [ ]:
c= 'TTT'
d = 'GGG'

In [ ]:
#l1,l2 = zip(*a)
b =f'INSERT INTO {db_table} (userid,songid,songlistindex,seedsongid,recdate) VALUES {*a,} ;'

In [ ]:
b

In [ ]:
l1,l2 = zip(*a)

### test insert bulk

In [16]:
time_table = {'get_user_ids': [],
              'get_user_song_id_source_genre': [],
              'get_acoustical_features': [],
              'get_acoustical_features': [],
              'get_popularity': [],
              'get_artist_id': [],
              'get_genres': [],
              'create_feature_object': [],
              'get_results': [],
              'filter_model': [], 
              'song_id_prediction_output': [],
              'db_connect': [],
              'insert_user_predictions': [],
              'get_stale_results': [],
              'get_stale_seed': []}

if FLASK_ENV == 'production':
    db_table = 'recommendations'
elif FLASK_ENV == 'development':
    db_table = 'recommendations_dev'

class Sound_Drip:
    

    def __init__(self, token):
        self.token = token
        self.sp = spotipy.Spotify(auth=self.token)
        self.user_id, self.display_name = self.get_user_ids()
        self.stale_seed_list = self.get_stale_seed()
        self.stale_results_list = self.get_stale_results()
        self.song_id, self.source_genre = self.get_user_song_id_source_genre()
        self.acoustical_features = self.get_acoustical_features(self.song_id)
        self.popularity = self.get_popularity(self.song_id)
        self.song_features_df = self.create_feature_object(
            self.popularity, self.acoustical_features)
        self.results = self.get_results(self.song_features_df)
        self.filtered_list = self.filter_model(self.results, self.source_genre)
        self.song_id_predictions = self.song_id_prediction_output(
            self.filtered_list)
        self.insert_user_predictions()#, print("predicts inserted into db")

    def get_user_ids(self):
        
        print("START TIME FOR get_user_ids")
        start_time = time.time()
        current_user_dict = self.sp.current_user()
        display_name = current_user_dict['display_name']
        user_id = current_user_dict['id']
        #print("retrieving user id and display name for current token")
        print(f"TOTAL get_user_ids TIME: {time.time() - start_time} seconds")
        time_table['get_user_ids'].append(time.time() - start_time)
        return user_id, display_name

    def get_user_song_id_source_genre(self):
        
        print("START TIME FOR get_user_song_id_source_genre")
        start_time = time.time()
        stale_songs = self.stale_seed_list
        results = self.sp.current_user_saved_tracks(limit=50)
        for song_number in range(0, len(results['items'])):
            #print(song_number)
            song_id = results['items'][song_number]['track']['id']
            #print(song_id)
            if song_id not in stale_songs:
                artist_id = self.get_artist_id(song_id)
                genre = self.get_genres(artist_id)
                #print(genre)
                if genre != []:
                    break
                else:
                    continue
            else:
                if song_number == len(results['items']) - 1:
                    #print("application out of fresh seeds")
                    for song_id in stale_songs:
                        artist_id = self.get_artist_id(song_id)
                        genre = self.get_genres(artist_id)
                        if genre != []:
                            break
                        else:
                            continue
        print(f" TOTAL get_user_song_id_source_genre TIME: {time.time() - start_time} seconds")
        time_table['get_user_song_id_source_genre'].append(time.time() - start_time)
        return song_id, genre

    def get_acoustical_features(self, song_id):
        
        print("START TIME FOR get_acoustical_features")
        start_time = time.time()
        acoustical_features = self.sp.audio_features(song_id)[0]
        print(f" TOTAL get_acoustical_features TIME: {time.time() - start_time} seconds")
        time_table['get_acoustical_features'].append(time.time() - start_time)
        return acoustical_features

    def get_popularity(self, song_id):
        
        print("START TIME FOR get_popularity")
        start_time = time.time()
        popularity = self.sp.track(song_id)['popularity']
        print(f" TOTAL get_popularity TIME: {time.time() - start_time} seconds")
        time_table['get_popularity'].append(time.time() - start_time)
        return popularity

    def get_artist_id(self, song_id):
        
        print("START TIME FOR get_artist_id")
        start_time = time.time()
        artist = self.sp.track(song_id)['artists'][0]['id']
        print(f" TOTAL get_artist_id TIME: {time.time() - start_time} seconds")
        time_table['get_artist_id'].append(time.time() - start_time)
        return artist

    def get_genres(self, artist):
        
        print("START TIME FOR get_genres")
        start_time = time.time()
        genre = self.sp.artist(artist)['genres']
        print(f" TOTAL get_genres TIME: {time.time() - start_time} seconds")
        time_table['get_genres'].append(time.time() - start_time)
        return genre

    def create_feature_object(self, popularity, acoustical_features):
        
        print("START TIME FOR create_feature_object")
        start_time = time.time()
        popularity_dict = {'popularity': popularity}
        song_features = acoustical_features
        song_features.update(popularity_dict)
        song_features = {
            "audio_features": {
                key: song_features[key] for key in song_features.keys() & {
                    'popularity',
                    'acousticness',
                    'danceability',
                    'energy',
                    'instrumentalness',
                    'key',
                    'liveness',
                    'loudness',
                    'mode',
                    'speechiness',
                    'tempo',
                    'time_signature',
                    'valence'}}}

        df = pd.DataFrame.from_dict(
            json_normalize(
                song_features["audio_features"]),
            orient='columns')
        df = df.reindex(sorted(df.columns), axis=1)
        print(f" TOTAL create_feature_object TIME: {time.time() - start_time} seconds")
        time_table['create_feature_object'].append(time.time() - start_time)
        return df

    def get_results(self, song_features_df):
        
        print("START TIME FOR get_results")
        start_time = time.time()
        scaler = load("./models/scalar3.joblib")
        #print('Scaling data...')
        data_scaled = scaler.transform(song_features_df)
        normalizer = Normalizer()
        data_normalized = normalizer.fit_transform(data_scaled)
        #print('Loading pickled model...')
        model = load('./models/model5.joblib')
        results = model.kneighbors([data_normalized][0])[1:]
        #print('results returned')
        print(f" TOTAL get_results TIME: {time.time() - start_time} seconds")
        time_table['get_results'].append(time.time() - start_time)
        return results[0]

    def filter_model(self, model_results, source_genre_list):
        
        # loop takes KNN results and filters by source track genres
        print("START TIME FOR filter_model")
        start_time = time.time()
        #print(source_genre_list)
        #print("filter for genres initiated")
        genre_array = pickle.load(open("./data/genres_array_2.pkl", "rb"))
        filtered_list = []
        song_list_length = 20
        stale_results = self.stale_results_list
        model_results_before = len(model_results[0][1:])
        model_results = [index for index in model_results[0]
                         [1:] if index not in stale_results]
        model_results_final = model_results_before - len(model_results)
        #print(f'{model_results_final} stale tracks were removed for the user')
        for output_song_index in model_results:
            output_genre_list = genre_array[output_song_index]
            for output_genre in output_genre_list:
                output_genre = output_genre.strip(" ")
                for source_genre in source_genre_list:
                    source_genre = "'" + source_genre + "'"
                    if source_genre == output_genre:
                        filtered_list.append(output_song_index)
                    else:
                        continue
        filtered_list = list(unique_everseen(filtered_list))
        if len(filtered_list) >= song_list_length:
            #print("filter found at least 20 genre matches")
            filtered_list = filtered_list[0:20]
        else:
            counter = song_list_length - len(filtered_list)
            #print("length of filtered list:", len(filtered_list))
            #print(f'need to add {counter} items to final song output')
            for output_song_index in model_results:
                if output_song_index not in filtered_list:
                    if counter > 0:
                        filtered_list.append(output_song_index)
                        counter -= 1
                    else:
                        break
        #print(
        #    f"filtered list with {len(filtered_list)} unique song indices returned")
        print(f" TOTAL filter_model TIME: {time.time() - start_time} seconds")
        time_table['filter_model'].append(time.time() - start_time)
        return filtered_list

    def song_id_prediction_output(self, filtered_list):
        
        print("START TIME FOR song_id_prediction_output")
        start_time = time.time()
        similar_songs = []
        song_id_list = []
        #print('song_id_list loading...')
        song_id_array = pickle.load(open('./data/song_id_array3.pkl', 'rb'))
        #print('song_id_list loaded')
        for song_row in filtered_list:
            song_id = song_id_array[song_row]
            similar_songs.append({'similarity': [.99], 'values': song_id})
            song_id_list.append(song_id)
        song_result_output_dict = {"songs": similar_songs}
        song_id_and_index_dict = {
            song_id: song_index for song_id,
            song_index in zip(
                song_id_list,
                filtered_list)}
        #print("Results returned")
        print(f" TOTAL song_id_prediction_output TIME: {time.time() - start_time} seconds")
        time_table['song_id_prediction_output'].append(time.time() - start_time)
        return song_result_output_dict, song_id_and_index_dict

    def db_connect(self):
        
        print("START TIME FOR db_connect")
        start_time = time.time()
        conn = ps.connect(host=POSTGRES_ADDRESS,
                          database=POSTGRES_DBNAME,
                          user=POSTGRES_USERNAME,
                          password=POSTGRES_PASSWORD,
                          port=POSTGRES_PORT)
        cur = conn.cursor()
        print(f" TOTAL db_connect TIME: {time.time() - start_time} seconds")
        time_table['db_connect'].append(time.time() - start_time)
        return conn, cur

    # def get_user_ids(self):
    #     '''
    #     Retrieves user id from Spotfiy API
    #     Returns user_id, and display_name (display_name is for the database)
    #     '''
    #     current_user_dict = self.sp.current_user()
    #     display_name = current_user_dict['display_name']
    #     user_id = current_user_dict['id']
    #     print("retrieving user id and display name for current token")
    #     return user_id, display_name

    def insert_user_predictions(self):
        
        print("START TIME FOR insert_user_predictions")
        start_time = time.time()
        insert_bulk = f'INSERT INTO {db_table} (userid,songid,songlistindex,seedsongid,recdate) VALUES '
        try:
            conn, cur = self.db_connect()
            for song_id, song_index in self.song_id_predictions[1].items():
                values_segment = f'(\'{self.user_id}\',\'{song_id}\',\'{song_index}\',\'{self.song_id}\',current_timestamp)'
                insert_bulk = insert_bulk  + values_segment +','
            insert_bulk = insert_bulk[:-1] + ';'
            cur.execute(insert_bulk)
            conn.commit()
            conn.close()
        except ps.DatabaseError as e:
            print(f'Error {e}')
            sys.exit(1)
        finally:
            if conn:
                conn.close()
        print(f" TOTAL insert_user_predictions TIME: {time.time() - start_time} seconds")
        time_table['insert_user_predictions'].append(time.time() - start_time)

    def get_stale_results(self):
        
        print("START TIME FOR get_stale_results")
        start_time = time.time()
        try:
            conn, cur = self.db_connect()
            query = f'SELECT DISTINCT (songlistindex) FROM {db_table} WHERE userid = \'{self.user_id}\';'
            cur.execute(query)
            query_results = cur.fetchall()
            stale_results_list = [index[0] for index in query_results]
        except ps.DatabaseError as e:
            print(f'Error {e}')
            sys.exit(1)
        finally:
            if conn:
                conn.close()
        print(f" TOTAL get_stale_results TIME: {time.time() - start_time} seconds")
        time_table['get_stale_results'].append(time.time() - start_time)
        return stale_results_list

    def get_stale_seed(self):
        print("START TIME FOR get_stale_seed")
        start_time = time.time()
        try:
            conn,cur = self.db_connect()
            query = f'SELECT DISTINCT (seedsongid) FROM {db_table} WHERE userid = \'{self.user_id}\' AND seedsongid is not null;'
            cur.execute(query)
            query_results = cur.fetchall()
            stale_results_list = [index[0] for index in query_results]
        except ps.DatabaseError as e:
            print(f'Error {e}')
            sys.exit(1)
        finally:
            if conn:
                conn.close()
        print(f" TOTAL get_stale_seed TIME: {time.time() - start_time} seconds")
        time_table['get_stale_seed'].append(time.time() - start_time)
        return stale_results_list


In [17]:
sd = Sound_Drip(token)

START TIME FOR get_user_ids
TOTAL get_user_ids TIME: 0.19444608688354492 seconds
START TIME FOR get_stale_seed
START TIME FOR db_connect
 TOTAL db_connect TIME: 0.8577322959899902 seconds
 TOTAL get_stale_seed TIME: 1.1028871536254883 seconds
START TIME FOR get_stale_results
START TIME FOR db_connect
 TOTAL db_connect TIME: 0.8002114295959473 seconds
 TOTAL get_stale_results TIME: 1.0480217933654785 seconds
START TIME FOR get_user_song_id_source_genre
START TIME FOR get_artist_id
 TOTAL get_artist_id TIME: 0.08632230758666992 seconds
START TIME FOR get_genres
 TOTAL get_genres TIME: 0.08023476600646973 seconds
 TOTAL get_user_song_id_source_genre TIME: 0.30763769149780273 seconds
START TIME FOR get_acoustical_features
 TOTAL get_acoustical_features TIME: 0.07823801040649414 seconds
START TIME FOR get_popularity
 TOTAL get_popularity TIME: 0.09165740013122559 seconds
START TIME FOR create_feature_object
 TOTAL create_feature_object TIME: 0.0019941329956054688 seconds
START TIME FOR get_